In [6]:
def tokenizer(citationString):
  import re

  """
    Remove URL/DOI from the annotated citation string
    And cleanup texts before URL/DOI tags
    TODO: Currently keeping out of URL/DOI. Will work on this later
  """
  urlExists = False
  doiExists = False
  urlordoi = None
  if "<URL>" in citationString:
    urlExists = True
    """
      re.findall [0] gets the matched string
      re.sub gives the text inside the tag
    """
    urlordoi = re.sub('<[^<]+>', "", re.findall('<URL>.+<\/URL>',citationString)[0]) 
    citationString = re.sub('<URL>.+<\/URL>', "", citationString)
  if "<DOI>" in citationString:
    doiExists = True
    urlordoi = re.sub('<[^<]+>', "", re.findall('<DOI>.+<\/DOI>',citationString)[0]) 
    citationString = re.sub('<DOI>.+<\/DOI>', "", citationString)

  # # We remove the text in front of DOI/URL. It generally starts after a "," or "."
  # iter = 0
  # if urlExists or doiExists:
  #   for char in reversed(citationString):    
  #     if re.search(r"[,.;()\"'?!*]", char):      
  #       break            
  #     iter = iter - 1
  #   if iter != 0:
  #     citationString = citationString[:iter]
  # print(citationString)
  tokens = []
  tags = []
  unprocessedTokens = []


  # charPattern = r"[\w\d]"
  # charAssociatePattern = r"[<?\/?\->]"
  # noCharPattern = r"[\W]"
  charPattern = r"[<\/>\w\d]"
  noCharPattern = r"[\/ ,.:;()\"'?!+@#{\[}\]\\$^&*-]" 
  endPtr = 0
  word = ""

  """
    Seperating non-word and words(with tags) at first 
    e.g. ['<author><family>Mehrinfar<family>', ',', ' ', '<given>Ramona<given>']
    [Processed them later]
  """
  tag = False
  while endPtr < len(citationString):
    # Keep everything inside the XML tag
    if citationString[endPtr] == '<':
      tag = True  

    if tag:
      word += citationString[endPtr]

      if citationString[endPtr] == '>':
        tag = False
    else:    
      # If it's not XML tag, look for char & non-char 
      if re.search(charPattern, citationString[endPtr]):
        word += citationString[endPtr]
      elif re.search(noCharPattern, citationString[endPtr]):
        if word is not "":
          unprocessedTokens.append(word)   
        unprocessedTokens.append(citationString[endPtr])
        word = ""

    endPtr = endPtr + 1

  #print(unprocessedTokens)

  """
    Process the token and tag list [Cleaning tokens from tags and creating tag list]    
  """

  title = False
  publisher = False
  containerTitle = False
  firstName = False
  givenName = False

  insideOfTag = False
  outsideOfTag = False

  xmlEndPattern = r"<\/[^>]*>(?![\s\S]*<\/[^>]*>)"
  xmlStartPattern = r"<[a-zA-Z-]+>"

  idx = 0
  while idx in range(len(unprocessedTokens)):
    #if not (re.search(xmlStartPattern, unprocessedTokens[idx]) and re.search(xmlEndPattern, unprocessedTokens[idx])):
    if re.search(xmlStartPattern, unprocessedTokens[idx]):
      outsideOfTag = False
    if re.search(xmlEndPattern, unprocessedTokens[idx-1]):
      outsideOfTag = True

    if unprocessedTokens[idx] == "-":                       
      tags.append('HYPHEN')  

    elif unprocessedTokens[idx] == "/":                       
      tags.append('BSLASH') 

    elif unprocessedTokens[idx] == "\\":                       
      tags.append('FSLASH') 

    elif unprocessedTokens[idx] == "?":                       
      tags.append('QSMARK')  

    elif unprocessedTokens[idx] == "!":                       
      tags.append('EXCLAMATION')  
      
    elif unprocessedTokens[idx] == ";":                       
      tags.append('SEMICOLON')    

    elif unprocessedTokens[idx] == "(":                       
      tags.append('PARENTHESIS')    
    
    elif unprocessedTokens[idx] == ")":                       
      tags.append('PARENTHESIS') 

    elif unprocessedTokens[idx] == "{":                       
      tags.append('PARENTHESIS')    
    
    elif unprocessedTokens[idx] == "}":                       
      tags.append('PARENTHESIS')  

    elif unprocessedTokens[idx] == "[":                       
      tags.append('PARENTHESIS')    
    
    elif unprocessedTokens[idx] == "]":                       
      tags.append('PARENTHESIS')  
    
    elif unprocessedTokens[idx] == ":":                       
      tags.append('COLON')  
    
    elif unprocessedTokens[idx] == ".":                       
      tags.append('PERIOD')      
    
    elif unprocessedTokens[idx] == ",":                       
      tags.append('COMMA')    
    
    #elif unprocessedTokens[idx] == " ":                       
    #  tags.append('SPACE')   

    elif unprocessedTokens[idx] == "p" or unprocessedTokens[idx] == "pp":
      tags.append('PP')
      
      title = False
      publisher = False
      containerTitle = False
      firstName = False
      givenName = False    
    
    elif outsideOfTag:
      #if tags[-1] == "B-OTHER" or tags[-2] == "B-OTHER" or tags[-3] == "B-OTHER":
      #  tags.append('I-OTHER')
      #elif tags[-1] == "I-OTHER" or tags[-2] == "I-OTHER":
      #  tags.append('I-OTHER')  
      #else:
      tags.append('O')

      title = False  
      publisher = False 
      containerTitle = False 
      firstName = False
      givenName = False
    # HOTFIX
    elif len(unprocessedTokens[idx]) == 1 and re.search(r"\W", unprocessedTokens[idx]): # Incase: Not a defined punctuation, but not [a-zA-Z] character either
      #tags.append('B-OTHER')
      #  if len(tags) > 2:
      #      if tags[-1] == "B-OTHER" or tags[-2] == "B-OTHER":
      #          tags.append('I-OTHER')
      #      elif tags[-1] == "I-OTHER" or tags[-2] == "I-OTHER":
      #          tags.append('I-OTHER')  
      #  else:
      #      tags.append('B-OTHER') 
         tags.append('O') 
    elif "publisher" in unprocessedTokens[idx] and not publisher:
      tags.append('B-PUBLISHER')
      
      title = False  
      publisher = True 
      containerTitle = False 
      firstName = False
      givenName = False

    elif "source" in unprocessedTokens[idx]:
      tags.append('B-SOURCE')
      
      title = False
      publisher = False
      containerTitle = False   
      firstName = False
      givenName = False 

    elif "ISSN" in unprocessedTokens[idx]:
      tags.append('B-ISSN')
      
      title = False
      publisher = False
      containerTitle = False   
      firstName = False
      givenName = False 
    
    elif "page" in unprocessedTokens[idx]:
      tags.append('B-PAGE')
      
      title = False
      publisher = False
      containerTitle = False   
      firstName = False
      givenName = False 
    elif "<issued><year>" in unprocessedTokens[idx] or "<year>" in unprocessedTokens[idx] or "<issued>" in unprocessedTokens[idx]:
      tags.append('B-DATE')
      
      title = False
      publisher = False
      containerTitle = False
      firstName = False
      givenName = False

    elif "<month>" in unprocessedTokens[idx] or "<issued><month>" in unprocessedTokens[idx]:
      tags.append('B-MONTH')
      
      title = False
      publisher = False
      containerTitle = False    
      firstName = False
      givenName = False

    elif "<issue>" in unprocessedTokens[idx]:
      tags.append('B-ISSUE')
      
      title = False
      publisher = False
      containerTitle = False    
      firstName = False
      givenName = False

    elif "volume" in unprocessedTokens[idx]:
      tags.append('B-VOL')
      
      title = False
      publisher = False
      containerTitle = False   
      firstName = False
      givenName = False 

    elif containerTitle is True: # If no other tag is found yet, consider the tokens as title 
      tags.append('I-CONTAINERTITLE')

      title = False
      publisher = False
      firstName = False
      givenName = False

    elif "container-title" in unprocessedTokens[idx]:
      tags.append('B-CONTAINERTITLE')    
      
      title = False
      publisher = False
      containerTitle = True
      firstName = False
      givenName = False

    # HOTFIX
    # "In" before the start of container-title is to be tagged as B-OTHER
    elif outsideOfTag: #re.search(xmlEndPattern, unprocessedTokens[idx-1]):
      tags.append('O')

      title = False
      publisher = False
      containerTitle = False  
      firstName = False
      givenName = False  

    elif title is True: # If no other tag is found yet, consider the tokens as title 
      tags.append('I-TITLE')
      publisher = False
      containerTitle = False
      firstName = False
      givenName = False

    elif "<title>" in unprocessedTokens[idx]:
      tags.append('B-TITLE')
      
      title = True
      publisher = False
      containerTitle = False
      firstName = False
      givenName = False

    elif publisher is True: # If no other tag is found yet, consider the tokens as publisher 
      tags.append('I-PUBLISHER')

      title = False    
      containerTitle = False
      firstName = False
      givenName = False

    # HOTFIX
    # If there's "and" around the names, it should be tagged as OTHER
    elif (firstName or givenName) and unprocessedTokens[idx] == "and":
      tags.append('O')  

      firstName = False
      givenName = False

    elif "family" in unprocessedTokens[idx]:
      tags.append('B-FN')
      
      title = False
      publisher = False
      containerTitle = False
      firstName = True
      givenName = False

    elif "given" in unprocessedTokens[idx] and not givenName:    
      tags.append('B-GN')
      
      title = False
      publisher = False
      containerTitle = False  
      firstName = False
      givenName = True  

    elif firstName is True: # If no other tag is found yet, consider the tokens as title 
      tags.append('I-FN')
      
      publisher = False
      containerTitle = False
      firstName = False
      givenName = False

    elif givenName is True: # If no other tag is found yet, consider the tokens as title 
      tags.append('I-GN')
      
      publisher = False
      containerTitle = False
      firstName = False
      title = False

    else: 
      # if tags[-1] == "B-OTHER" or tags[-2] == "B-OTHER":
      #   tags.append('I-OTHER')  
      # else:
      tags.append('O') # Like "and" or words without tags

    # Remove xml tags from each token and keep only the word 
    tokens.append(re.sub('<[^<]+>', "", unprocessedTokens[idx]))
      
    idx = idx + 1

  """
    Add URL/DOI to a single token
    and add a Tag for that to the end of the list
    TODO: Will work on this later
  """
  # if urlExists:
  #   tags.append('B-URL')
  #   tokens.append(urlordoi)
  # elif doiExists:
  #   tags.append('B-DOI')
  #   tokens.append(urlordoi)

  #print(tokens)
  #print(tags)
  #print("Tokens  Tags")

  """
    Remove tokens like "<given>" and corresponding tag combination
  """
  idx = 0
  while idx in range(len(tokens)):
    if tokens[idx] == "":
      tokens.pop(idx)
      tags.pop(idx)
    #print(tokens[idx]+ "  "+ tags[idx])

    idx = idx + 1

  return (tokens, tags)

In [3]:
""" 
    Parallel process the dataset for building training data
    Utilize n cores and chunk the task into n pieces [n = n_proc]
"""
# import necessary libraries
import pandas as pd
import os
import glob
import numpy as np
import multiprocessing
import string
import random
import time

def getDataTuples(dataset):
    """
        Input# dataset: n rows of annotated ciation strings
        
        Process# Prepare each citation string using tokenizer mothod which returns word-label list
                 from a annotated citation string
        
        Output# List(List(word-lable tuple of one citation string))
    """
    #print('Chunked dataset before processing: {}'.format(dataset.shape))
    '''Read the dataset'''
    
    ''' Prepare dataframe for tuples'''    
    df2 = pd.DataFrame(columns=['Tokens'])
    df3 = pd.DataFrame(columns=['Tags'])
    for index, row in dataset.iterrows():
        tokenizerResponse = tokenizer(row["citationStringAnnotated"].strip())
        df2 = df2.append({'Tokens': tokenizerResponse[0]}, ignore_index=True)
        df3 = df3.append({'Tags': tokenizerResponse[1]}, ignore_index=True)        

    
    data = pd.concat([df2,df3],axis=1)
    #print('Chunked dataset | data: {}'.format(data.shape))

    
    ''' Prepare list of tuples'''
    datatuple = []
    for index, row in data.iterrows():
        tuples = []
        for idx in range(len(row['Tokens'])):
            # Yeilding out the spaces
            if row['Tokens'][idx] is not ' ':
                tuples.append((row['Tokens'][idx], row['Tags'][idx]))
        datatuple.append(tuples)
    
    #print('Chunked dataset after processing: {}'.format(len(datatuple)))
    return datatuple


def multiprocessing_func(x):
    """
        Each process creates a text file and write a chunk of processed data
        into that file.
    """
    print('Chunked dataset shape in this process: {}'.format(x.shape))
    length = 5 # length of filename
    randomstr = ''.join(random.choices(string.ascii_letters+string.digits,k=length))
    filename = randomstr +'.txt'
    filepath = "/home/muddi004/muddi/GIANT/data-downsized" + filename
    ''' Use datatuples to write into text file'''
    datatuple = getDataTuples(x)
    print('Datatuple {}'.format(len(datatuple)))
    with open(filepath, 'a+') as fileValid:  
        for sentence in datatuple:
            for tuples in sentence:                
                fileValid.write("{}\t{}\n".format(tuples[0],tuples[1]))    
            fileValid.write("\n")
    
    print('Done Writing {}'.format(filename))


# use glob to get all the csv files 
# in the folder
csvDirectory = r"/home/muddi004/muddi/GIANT/downsized-csv"
csv_files = glob.glob(os.path.join(csvDirectory, "*.csv"))

start = time.time()
print("Starting now")
for csv in csv_files[200:]:
    print("Now working on: {}".format(csv))
    dataset = pd.read_csv(csv)
    #dataset = dataset.sample(n=250000) # Random sampling to 250000 data
    n_proc = 40 # TODO: Change number of available processors
    chunks = np.array_split(dataset, n_proc)

    processes=[] #Initialize the parallel processes list
    for i in np.arange(0,n_proc):    
        """Execute the target function on the n_proc target processors using the splitted input""" 
        p = multiprocessing.Process(target=multiprocessing_func,args=(chunks[i],))
        processes.append(p)
        p.start()
    for process in processes:
        process.join()

end = time.time()
print("Total time required: {}".format(end - start))

Starting now
Now working on: /home/muddi004/muddi/GIANT/downsized-csv/XDPDT.csv
Chunked dataset shape in this process: (6250, 5)Chunked dataset shape in this process: (6250, 5)

Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process: (6250, 5)
Chunked dataset shape in this process:

In [2]:
""" Parallel process the dataset for building validation data"""
# import necessary libraries
import pandas as pd
import os
import glob
import numpy as np
import multiprocessing
import string
import random
import time

def getDataTuples(dataset):
    #print('Chunked dataset before processing: {}'.format(dataset.shape))
    '''Read the dataset'''
    
    ''' Prepare dataframe for tuples'''    
    df2 = pd.DataFrame(columns=['Tokens'])
    df3 = pd.DataFrame(columns=['Tags'])
    for index, row in dataset.iterrows():
        tokenizerResponse = tokenizer(row["citationStringAnnotated"].strip())
        df2 = df2.append({'Tokens': tokenizerResponse[0]}, ignore_index=True)
        df3 = df3.append({'Tags': tokenizerResponse[1]}, ignore_index=True)        

    
    data = pd.concat([df2,df3],axis=1)
    #print('Chunked dataset | data: {}'.format(data.shape))

    
    ''' Prepare list of tuples'''
    datatuple = []
    for index, row in data.iterrows():
        tuples = []
        for idx in range(len(row['Tokens'])):
            # Yeilding out the spaces
            if row['Tokens'][idx] is not ' ':
                tuples.append((row['Tokens'][idx], row['Tags'][idx]))
        datatuple.append(tuples)
    
    #print('Chunked dataset after processing: {}'.format(len(datatuple)))
    return datatuple

def multiprocessing_func(x):
    print('Chunked dataset shape in this process: {}'.format(x.shape))
    length = 5 # length of filename
    randomstr = ''.join(random.choices(string.ascii_letters+string.digits,k=length))
    filename = randomstr +'.txt'
    filepath = "/home/muddi004/muddi/GIANT/data-validation/" + filename
    ''' Use datatuples to write into text file'''
    datatuple = getDataTuples(x)
    print('Datatuple {}'.format(len(datatuple)))
    with open(filepath, 'a+') as fileValid:  
        for sentence in datatuple:
            for tuples in sentence:                
                fileValid.write("{}\t{}\n".format(tuples[0],tuples[1]))    
            fileValid.write("\n")
    
    print('Done Writing {}'.format(filename))


# use glob to get all the csv files 
# in the folder
csvDirectory = r"/home/muddi004/muddi/GIANT/data-validation/"
csv_files = glob.glob(os.path.join(csvDirectory, "*.csv"))

start = time.time()
print("Starting now")
for csv in csv_files:
    print("Now working on: {}".format(csv))
    dataset = pd.read_csv(csv)
    #dataset = dataset.sample(n=250000) # Random sampling to 250000 data
    n_proc = 40 # TODO: Change number of available processors
    chunks = np.array_split(dataset, n_proc)

    processes=[] #Initialize the parallel processes list
    for i in np.arange(0,n_proc):    
        """Execute the target function on the n_proc target processors using the splitted input""" 
        p = multiprocessing.Process(target=multiprocessing_func,args=(chunks[i],))
        processes.append(p)
        p.start()
    for process in processes:
        process.join()

end = time.time()
print("Total time required: {}".format(end - start))

Starting now
Now working on: /home/muddi004/muddi/GIANT/data-validation/output_sampleCrossref.csv
Chunked dataset shape in this process: (391, 5)

Chunked dataset shape in this process: (391, 5)Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)Chunked dataset shape in this process: (391, 5)Chunked dataset shape in this process: (391, 5)


Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process: (391, 5)Chunked dataset shape in this process: (391, 5)


Chunked dataset shape in this process: (391, 5)
Chunked dataset shape in this process

In [3]:
""" Parallel process the dataset for building Test data"""
# import necessary libraries
import pandas as pd
import os
import glob
import numpy as np
import multiprocessing
import string
import random
import time

def getDataTuples(dataset):
    #print('Chunked dataset before processing: {}'.format(dataset.shape))
    '''Read the dataset'''
    
    ''' Prepare dataframe for tuples'''    
    df2 = pd.DataFrame(columns=['Tokens'])
    df3 = pd.DataFrame(columns=['Tags'])
    for index, row in dataset.iterrows():
        tokenizerResponse = tokenizer(row["citationStringAnnotated"].strip())
        df2 = df2.append({'Tokens': tokenizerResponse[0]}, ignore_index=True)
        df3 = df3.append({'Tags': tokenizerResponse[1]}, ignore_index=True)        

    
    data = pd.concat([df2,df3],axis=1)
    #print('Chunked dataset | data: {}'.format(data.shape))

    
    ''' Prepare list of tuples'''
    datatuple = []
    for index, row in data.iterrows():
        tuples = []
        for idx in range(len(row['Tokens'])):
            # Yeilding out the spaces
            if row['Tokens'][idx] is not ' ':
                tuples.append((row['Tokens'][idx], row['Tags'][idx]))
        datatuple.append(tuples)
    
    #print('Chunked dataset after processing: {}'.format(len(datatuple)))
    return datatuple

def multiprocessing_func(x):
    print('Chunked dataset shape in this process: {}'.format(x.shape))
    length = 5 # length of filename
    randomstr = ''.join(random.choices(string.ascii_letters+string.digits,k=length))
    filename = randomstr +'.txt'
    filepath = "/home/muddi004/muddi/GIANT/data/" + 'data-test.txt'
    ''' Use datatuples to write into text file'''
    datatuple = getDataTuples(x)
    print('Datatuple {}'.format(len(datatuple)))
    with open(filepath, 'a+') as fileValid:  
        for sentence in datatuple:
            for tuples in sentence:                
                fileValid.write("{}\t{}\n".format(tuples[0],tuples[1]))    
            fileValid.write("\n")
    
    print('Done Writing {}'.format(filename))


# use glob to get all the csv files 
# in the folder
csvDirectory = r"/home/muddi004/muddi/GIANT/data-test/"
csv_files = glob.glob(os.path.join(csvDirectory, "*.csv"))

start = time.time()
print("Starting now")
for csv in csv_files:
    print("Now working on: {}".format(csv))
    dataset = pd.read_csv(csv)
    #dataset = dataset.sample(n=250000) # Random sampling to 250000 data
    n_proc = 1 # TODO: Change number of available processors
    chunks = np.array_split(dataset, n_proc)

    processes=[] #Initialize the parallel processes list
    for i in np.arange(0,n_proc):    
        """Execute the target function on the n_proc target processors using the splitted input""" 
        p = multiprocessing.Process(target=multiprocessing_func,args=(chunks[i],))
        processes.append(p)
        p.start()
    for process in processes:
        process.join()

end = time.time()
print("Total time required: {}".format(end - start))

Starting now
Now working on: /home/muddi004/muddi/GIANT/data-test/test.csv
Chunked dataset shape in this process: (201, 2)
Datatuple 201
Done Writing 3nsnS.txt
Total time required: 1.1635541915893555


In [4]:
""" 
    Random Testing block 

"""

import time
import glob
import numpy as np
import pandas as pd


print("Starting")
start = time.time()
csvDirectory = r"/home/muddi004/muddi/GIANT/"
csv_files = glob.glob(os.path.join(csvDirectory, r"*.csv"))
filecount = 5
for csv in csv_files[filecount:]:
    print("Now working on file-no:{} name:{}".format(filecount+1,csv))
    dataset = pd.read_csv(csv)
    print("Original Dataset shape: {}".format(dataset.shape))
    dataset = dataset.sample(n=250000)
    print("Sample dataset shape: {}".format(dataset.shape))
    #n_proc = 40 # TODO: Change number of available processors
    #chunks = np.array_split(dataset, n_proc)
    #print("First chunk size:{}".format(chunks[0].shape))
    filecount = filecount + 1
    
end = time.time()
print("Total time required: {}".format(end - start))

Starting
Now working on file-no:6 name:/home/muddi004/muddi/GIANT/1 Billion Citation Dataset, v1 (61).csv
Original Dataset shape: (4563117, 5)
Sample dataset shape: (250000, 5)
Now working on file-no:7 name:/home/muddi004/muddi/GIANT/1 Billion Citation Dataset, v1 (163).csv
Original Dataset shape: (4547341, 5)
Sample dataset shape: (250000, 5)
Now working on file-no:8 name:/home/muddi004/muddi/GIANT/1 Billion Citation Dataset, v1 (175).csv
Original Dataset shape: (4523873, 5)
Sample dataset shape: (250000, 5)
Now working on file-no:9 name:/home/muddi004/muddi/GIANT/1 Billion Citation Dataset, v1 (134).csv
Original Dataset shape: (4527091, 5)
Sample dataset shape: (250000, 5)
Now working on file-no:10 name:/home/muddi004/muddi/GIANT/1 Billion Citation Dataset, v1 (176).csv
Original Dataset shape: (4528469, 5)
Sample dataset shape: (250000, 5)
Now working on file-no:11 name:/home/muddi004/muddi/GIANT/1 Billion Citation Dataset, v1 (214).csv
Original Dataset shape: (4509858, 5)
Sample dat

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
"""
    Random Testing Block
"""

# import necessary libraries
import pandas as pd
import os
import glob
import numpy as np
import multiprocessing
import string
import random
import time

# use glob to get all the csv files 
# in the folder
test1 = "/home/muddi004/muddi/GIANT/data-test/test-1.csv"
test2 = "/home/muddi004/muddi/GIANT/data-test/test-2.csv"
csv_files = [test1,test2]#glob.glob(os.path.join(r"/home/muddi004/muddi/GIANT/data-test/", "*.csv"))
#dataset = pd.read_csv(csvDirectory, encoding= 'unicode_escape')
print(csv_files)
#print(csv_files[:2])

# loop over the list of csv files
'''
print(dataset.shape)
datatuple = []
dataChunks = np.array_split(dataset, 200)
#print(dataChunks[1].head)
csv_files = dataChunks
'''

for f in csv_files:      
    # read the csv file
    print(f)

    df = pd.read_csv(f, encoding='unicode_escape')
    print(df.shape)
     
    # print the location and filename
    #print('Location:', f)
    #print('File Name:', f.split("\\")[-1])
      
    # print the content
    #print('Content:')
    #display(df)
    

    df2 = pd.DataFrame(columns=['Tokens'])
    df3 = pd.DataFrame(columns=['Tags'])
    datatuple = []
    for index, row in df.iterrows():
        print("Now on: {}".format(row["citationStringAnnotated"]))
        tokenizerResponse = tokenizer(row["citationStringAnnotated"].strip())

        df2 = df2.append({'Tokens': tokenizerResponse[0]}, ignore_index=True)
        df3 = df3.append({'Tags': tokenizerResponse[1]}, ignore_index=True)

        data = pd.concat([df2,df3],axis=1)
        #data.head()

        for index, row in data.iterrows():
            tuples = []
            for idx in range(len(row['Tokens'])):
                # Yeilding out the spaces
                if row['Tokens'][idx] is not ' ':
                    tuples.append((row['Tokens'][idx], row['Tags'][idx]))
        datatuple.append(tuples)
        
    length = 5 # length of filename
    randomstr = ''.join(random.choices(string.ascii_letters+string.digits,k=length))
    filename = randomstr +'.txt'
    filepath = "/home/muddi004/muddi/GIANT/data-test/" + filename
    ''' Use datatuples to write into text file'''
    print('Datatuple {}'.format(len(datatuple)))
    with open(filepath, 'a+') as fileValid:  
        for sentence in datatuple:
            for tuples in sentence:                
                fileValid.write("{}\t{}\n".format(tuples[0],tuples[1]))    
            fileValid.write("\n")

    print('Done Writing {}'.format(filename))
  